# Project 2 : Two dimentional Ising model

In [1]:
import pickle
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as skms
import sklearn.linear_model as skl
import sklearn.metrics as skm
import tqdm
import copy
import time
from IPython.display import display

%matplotlib inline

sns.set(color_codes=True)

import algorithms

In [2]:
cwd = os.getcwd()
filenames = glob.glob(os.path.join(cwd, 'files*'))

In [3]:
label_filename = "/home/hanna/project2/files/Ising2DFM_reSample_L40_T=All_labels.pkl"

# Read in the labels
with open(label_filename, "rb") as f:
    labels = pickle.load(f)

dat_filename = "/home/hanna/project2/files/Ising2DFM_reSample_L40_T=All.pkl"
    
# Read in the corresponding configurations
with open(dat_filename, "rb") as f:
    data = np.unpackbits(pickle.load(f)).reshape(-1, 1600).astype("int")

# Set spin-down to -1
data[data == 0] = -1

In [4]:
data.shape

(160000, 1600)

In [5]:
# Set up slices of the dataset
ordered = slice(0, 70000)
critical = slice(70000, 100000)
disordered = slice(100000, 160000)

X = np.concatenate((data[ordered], data[disordered]))
Y = np.concatenate((labels[ordered], labels[disordered]))

In [6]:
labels[ordered].shape, labels[disordered].shape

((70000,), (60000,))

In [7]:
data[ordered].shape, data[disordered].shape

((70000, 1600), (60000, 1600))

In [8]:
Y.shape

(130000,)

In [9]:
data[disordered]

array([[ 1,  1, -1, ..., -1,  1,  1],
       [ 1,  1, -1, ...,  1,  1,  1],
       [-1, -1, -1, ..., -1, -1, -1],
       ..., 
       [ 1,  1,  1, ...,  1,  1, -1],
       [ 1,  1,  1, ...,  1, -1, -1],
       [ 1,  1,  1, ...,  1, -1, -1]])

In [10]:
X.shape

(130000, 1600)

In [11]:
X_train, X_test, y_train, y_test = skms.train_test_split(X,Y, test_size = 0.333)

In [12]:
from utils import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, split_size=0.5, random_state=0)

In [13]:
# Testing that y_test and train contain both label 
y_train, y_test

(array([1, 0, 0, ..., 1, 1, 1]), array([1, 1, 1, ..., 0, 0, 0]))

# Sammenlign med Stochastic gradient descent. 

In [14]:
n_iter = 50
random_state = 1
key = "ridge" 
descent_method= "sgd"

eta = [0.0001, 0.001, 0.01, 0.1]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]

accu = []

for e in eta:
    for l in lmd:
        a = algorithms.LogisticRegression(eta = e, random_state = random_state, 
                                          key = key, dm = descent_method, 
                                          shuffle = True, n_iter = 100, 
                                          batch_size = 100, epochs=10,lmd = l, 
                                          tolerance=1e-14)
        log = a.fit(X_train, y_train)
        score = log.predict(X_test)
        acc = np.sum(score == y_test)/len(score)
        accu.append(acc)
        print(acc)

KeyboardInterrupt: 

In [ ]:
accu # prøv ny batch størrelse

In [ ]:
m = np.array(accu).reshape((len(eta), len(lmd)))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=lmd, yticklabels=eta)
ax.set_title("Logistic regression SGD Ridge", fontsize = 20)
ax.set_xlabel("Lambda value", fontsize = 15)
ax.set_ylabel("Eta value", fontsize = 15) #
plt.savefig("./results/figures/Logisticregression_Ridge_sgd.png")

In [ ]:
n_iter = 50
random_state = 1
key = "lasso" 
descent_method= "sgd"

eta = [0.0001, 0.001, 0.01, 0.1]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]

accu = []

for e in eta:
    for l in lmd:
        a = algorithms.LogisticRegression(eta = e, random_state = random_state, 
                                          key = key, dm = descent_method, 
                                          shuffle = True, n_iter = 100, 
                                          batch_size = 10, epochs=100,lmd = l, 
                                          tolerance=1e-14)
        log = a.fit(X_train, y_train)
        score = log.predict(X_test)
        acc = np.sum(score == y_test)/len(score)
        accu.append(acc)

In [ ]:
m = np.array(accu).reshape((len(eta), len(lmd)))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=lmd, yticklabels=eta)
ax.set_title("Logistic regression SGD Ridge", fontsize = 20)
ax.set_xlabel("Lambda value", fontsize = 15)
ax.set_ylabel("Eta value", fontsize = 15) #
plt.savefig("./results/figures/Logisticregression_Lasso_sgd.png")

In [ ]:
random_state = 1
key = "ols" 
descent_method= "sgd"

eta = [0.0001, 0.001, 0.01, 0.1]
#lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]

accu = []

for e in eta:
 #   for l in lmd:
    a = algorithms.LogisticRegression(eta = e, random_state = random_state, 
                                      key = key, dm = descent_method, 
                                      shuffle = True, n_iter = 100, 
                                      batch_size = 10, epochs=100,lmd = 0, 
                                      tolerance=1e-14)
    log = a.fit(X_train, y_train)
    score = log.predict(X_test)
    acc = np.sum(score == y_test)/len(score)
    accu.append(acc)

In [ ]:
m = np.array(accu).reshape((len(eta), 1))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=lmd, yticklabels=eta)
ax.set_title("Logistic regression SGD Ridge", fontsize = 20)
ax.set_xlabel("Lambda value", fontsize = 15)
ax.set_ylabel("Eta value", fontsize = 15) #
plt.savefig("./results/figures/Logisticregression_OLS_sgd.png")

# Standard/steepest gradient descent

In [ ]:
eta = 0.01
n_iter = 50
random_state = 1
key = "ols" 
lmd=0.01
# (self, eta, random_state, key, n_iter = 50, lmd = 0, tolerance=1e-14):
a = algorithms.LogisticRegression(eta = e, random_state = random_state, 
                                  key = key, dm = "steepest", 
                                  shuffle = True, n_iter = 100, 
                                  batch_size = 100, epochs=10,lmd = l, 
                                  tolerance=1e-14)

In [ ]:
log = a.fit(X_train, y_train)

In [ ]:
score = log.predict(X_test)

In [ ]:
score

In [ ]:
acc = np.sum(score == y_test)/len(score)
acc

In [ ]:
accuracy = {"ols": [], "ridge":[], "lasso": []}

In [ ]:
eta = [0.0001, 0.001, 0.01, 0.1]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]
key = "lasso"

for e in eta:
    for l in lmd:
        a = algorithms.LogisticRegression(e, random_state, key, n_iter, l)
        log = a.fit(X_train, y_train)
        score = log.predict(X_test)
        acc = np.sum(score == y_test)/len(score)
        accuracy[key].append(acc)

In [ ]:
accuracy["lasso"]

In [ ]:
m = np.array(accuracy["lasso"]).reshape((len(eta), len(lmd)))
m

In [ ]:
m = np.array(accuracy["lasso"]).reshape((len(eta), len(lmd)))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=lmd, yticklabels=eta)
ax.set_title("Accuracy of prediction using Lasso", fontsize = 20)
ax.set_xlabel("Lambda value", fontsize = 15)
ax.set_ylabel("Eta value", fontsize = 15)
plt.savefig("./results/figures/Logisticregression_Lasso.png")

# OLS

In [ ]:
key = "ols"
etas = [0.0001, 0.001, 0.01, 0.1]

for e in etas:
    a = algorithms.LogisticRegression(e, random_state, key, n_iter)
    log = a.fit(X_train, y_train)
    score = log.predict(X_test)
    acc = np.sum(score == y_test)/len(score)
    accuracy[key].append(acc)

In [ ]:
m = np.array(accuracy[key])
print(m)
#np.array(m).reshape((4, 1))

In [ ]:
m = np.array(accuracy[key]).reshape(1,(len(eta)))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=eta)
ax.set_title("Logistic regression OLS", fontsize = 20)
ax.set_xlabel("Eta value", fontsize = 15)
ax.set_ylabel("Lambda value (lmd=0)", fontsize = 15)
plt.savefig("./results/figures/Logisticregression_OLS.png")

# Ridge

In [ ]:
eta = [0.0001, 0.001, 0.01, 0.1]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]

key = "ridge"

for e in eta:
    for l in lmd:
        a = algorithms.LogisticRegression(e, random_state, key, n_iter, l)
        log = a.fit(X_train, y_train)
        score = log.predict(X_test)
        acc = np.sum(score == y_test)/len(score)
        accuracy[key].append(acc)

In [ ]:
m = np.array(accuracy[key]).reshape((len(eta), len(lmd)))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=lmd, yticklabels=eta)
ax.set_title("Logistic regression Ridge", fontsize = 20)
ax.set_xlabel("Lambda value", fontsize = 15)
ax.set_ylabel("Eta value", fontsize = 15) #
plt.savefig("./results/figures/Logisticregression_Ridge.png")

Accuracy of one can be reffered to as a perfect classyfier.
# OBS! Sammenlign dette med scikitlearn

In [ ]:
# Case ridge for the best? optimizer.
clf = skl.LogisticRegression()
clf.fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test, y_test)

In [ ]:
# Should be similar to lasso with the best optimizer.
clf = skl.LogisticRegression(penalty='l1')
clf.fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test, y_test)